<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/Emotionline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emitionline 데이터셋

In [9]:
! tar -xvzf /content/drive/My\ Drive/Datasets/Emotionlines/EmotionLines_friends_annotation.tar.gz

EmotionLines/Friends/friends_train.json
EmotionLines/Friends/friends_dev.json
EmotionLines/Friends/friends_test.json
EmotionLines/README


In [50]:
import pandas as pd
import json

In [58]:
with open('/content/EmotionLines/Friends/friends_train.json', encoding='utf-8', mode='r') as f:
  array = json.load(f)

data =[]
for i in range(len(array)):
  data += array[i]
print(f"Total length of data : {len(data)}")
df = pd.DataFrame(data)

Total length of data : 10561


In [59]:
df

,speaker,utterance,emotion,annotation
0,Chandler,also I was the point person on my companys tr...,neutral,4100000
1,The Interviewer,You mustve had your hands full.,neutral,5000000
2,Chandler,That I did. That I did.,neutral,5000000
3,The Interviewer,So lets talk a little bit about your duties.,neutral,5000000
4,Chandler,My duties? All right.,surprise,2000030
...,...,...,...,...
10556,Chandler,You or me?,neutral,3000011
10557,Ross,"I got it. Uh, Joey, women don't have Adam's ap...",non-neutral,2100011
10558,Joey,"You guys are messing with me, right?",surprise,0000050
10559,All,Yeah.,neutral,4000010


In [53]:
df['emotion'].value_counts()

neutral        4752
non-neutral    2017
joy            1283
surprise       1220
anger           513
sadness         351
disgust         240
fear            185
Name: emotion, dtype: int64

In [78]:
# change the value to dtype: Category
df['emotion'] = pd.Categorical(df['emotion'])

# cat(egory).codes print the code of category
df['emotion'] = df.emotion.cat.codes

In [79]:
df.emotion

0        4
1        4
2        4
3        4
4        7
        ..
10556    4
10557    5
10558    7
10559    4
10560    5
Name: emotion, Length: 10561, dtype: int8

In [57]:
df['emotion'].value_counts()

4    4752
5    2017
3    1283
7    1220
0     513
6     351
1     240
2     185
Name: emotion, dtype: int64

In [87]:
for i, utter in enumerate(df.utterance):
  if i < 5:
    print(utter)
  else:
    break

also I was the point person on my companys transition from the KL-5 to GR-6 system.
You mustve had your hands full.
That I did. That I did.
So lets talk a little bit about your duties.
My duties?  All right.


# Tensorflow

make the dataset to tensor

In [91]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np

In [ ]:
target = df.pop('utterance')

In [97]:
target

0        also I was the point person on my companys tr...
1                         You mustve had your hands full.
2                                  That I did. That I did.
3            So lets talk a little bit about your duties.
4                                   My duties?  All right.
                               ...                        
10556                                           You or me?
10557    I got it. Uh, Joey, women don't have Adam's ap...
10558                 You guys are messing with me, right?
10559                                                Yeah.
10560    That was a good one. For a second there, I was...
Name: utterance, Length: 10561, dtype: object

In [99]:
df.values

array([['Chandler', 4, '4100000'],
       ['The Interviewer', 4, '5000000'],
       ['Chandler', 4, '5000000'],
       ...,
       ['Joey', 7, '0000050'],
       ['All', 4, '4000010'],
       ['Joey', 5, '1200020']], dtype=object)

In [100]:
dataset = tf.data.Dataset.from_tensor_slices(target)

In [106]:
for t in dataset.take(5):
  print(t)

tf.Tensor(b'also I was the point person on my company\xc2\x92s transition from the KL-5 to GR-6 system.', shape=(), dtype=string)
tf.Tensor(b'You must\xc2\x92ve had your hands full.', shape=(), dtype=string)
tf.Tensor(b'That I did. That I did.', shape=(), dtype=string)
tf.Tensor(b'So let\xc2\x92s talk a little bit about your duties.', shape=(), dtype=string)
tf.Tensor(b'My duties?  All right.', shape=(), dtype=string)


In [108]:
train_data = dataset.shuffle(len(df))
for t in train_data.take(5):
  print(t)

tf.Tensor(b'Nothing, nothing.', shape=(), dtype=string)
tf.Tensor(b'Yeah, I mean whatever.', shape=(), dtype=string)
tf.Tensor(b'Sounds nice.', shape=(), dtype=string)
tf.Tensor(b'Ohhh, I', shape=(), dtype=string)
tf.Tensor(b'Good runnin\xc2\x92 into you at the bank today, so ah, here\xc2\x92s my number, 555-9323. Give me a call. Later.', shape=(), dtype=string)


# Tokenizer

In [121]:
tokenizer = K.preprocessing.text.Tokenizer(char_level=False)

In [124]:
tokenizer.fit_on_texts("Hello Hello I'm Josoo. I drank Helium gas and became like this")